In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
from fastapi import FastAPI, Query
from pydantic import BaseModel
from typing import List
import joblib

In [2]:
# Ruta al archivo Parquet
ruta_archivo_parquet_sitios = r'C:\Users\guard\OneDrive\Desktop\Henry Data Science\Proyecto-FInal\Datos\gm_sitios_NJNY_202502190828.parquet'

# Leer el archivo Parquet en un DataFrame
df = pd.read_parquet(ruta_archivo_parquet_sitios)

# Mostrar las columnas disponibles
print("Columnas disponibles en el DataFrame:")
print(df.columns)

# Mostrar las primeras filas del DataFrame para tener una idea del contenido
print("\nPrimeras filas del DataFrame:")
print(df.head())

Columnas disponibles en el DataFrame:
Index(['gmap_id', 'name', 'street_address', 'city', 'state', 'zip_code',
       'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'price',
       'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday', 'Delivery', 'Dine-in', 'Takeout', 'Good for kids', 'Casual',
       'Dinner', 'Lunch'],
      dtype='object')

Primeras filas del DataFrame:
                                     gmap_id                  name  \
index                                                                
0       0x4ccab4b3af9c7c85:0x9defa19fe4e295c          Alpine Pizza   
26     0x89de5f9627d4e973:0x5b0cbc68d57ffefb       Stewart's Shops   
34     0x89def3a6bb7c4091:0xde094955651ec4bf         Al's Pizzeria   
53     0x89c3acab12c10d5d:0x73285fc4bd781796        Gulistan Pizza   
59     0x89c259f744b20deb:0xb536eb5ee402f92f  FREEHOLD In The Park   

          street_address          city state zip_code   latitude  longitude  \
index        

In [3]:
# Convertir horarios a formato datetime y crear columnas binarias para horarios abiertos
dias_semana = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
horarios = {}
for dia in dias_semana:
    df[dia] = df[dia].fillna('Closed')
    horarios[dia] = df[dia].apply(lambda x: [t.strip() for t in x.split('-')] if x != 'Closed' and '-' in x else ['Closed', 'Closed'])

# Crear un DataFrame binario para los horarios
for dia in dias_semana:
    df[dia + '_open'] = horarios[dia].apply(lambda x: 1 if x[0] != 'Closed' else 0)
    df[dia + '_close'] = horarios[dia].apply(lambda x: 1 if x[1] != 'Closed' else 0)

# Seleccionar solo las columnas relevantes
columnas_binarias = ['Delivery', 'Dine-in', 'Takeout', 'Good for kids', 'Casual', 'Dinner', 'Lunch']
df_binarias = df[['name', 'avg_rating', 'num_of_reviews', 'latitude', 'longitude', 'zip_code'] + columnas_binarias + [dia + '_open' for dia in dias_semana] + [dia + '_close' for dia in dias_semana]]

# Guardar el DataFrame preprocesado
df_binarias.to_csv('data_preprocesada.csv', index=False)


In [4]:
# Cargar datos preprocesados
df_binarias = pd.read_csv('data_preprocesada.csv')

# Entrenar modelo Nearest Neighbors
X = df_binarias.drop(columns=['name', 'avg_rating', 'num_of_reviews', 'latitude', 'longitude', 'zip_code'])
modelo_knn = NearestNeighbors(n_neighbors=3, algorithm='auto').fit(X)

# Función para recomendar locales
def recomendar_locales(horario, condiciones_binarias, lat, lon):
    # Crear vector de consulta
    consulta = pd.Series([horario] + condiciones_binarias)
    consulta = consulta.values.reshape(1, -1)
    
    # Encontrar vecinos más cercanos
    distancias, indices = modelo_knn.kneighbors(consulta)
    recomendaciones = df_binarias.iloc[indices[0]]
    
    # Ordenar por num_of_reviews y avg_rating
    recomendaciones = recomendaciones.sort_values(by=['num_of_reviews', 'avg_rating'], ascending=[False, False])
    return recomendaciones[['name', 'avg_rating', 'num_of_reviews', 'latitude', 'longitude', 'zip_code']]


In [5]:
# Guardar el modelo
joblib.dump(modelo_knn, 'modelo_knn.pkl')

['modelo_knn.pkl']

In [6]:
# Inicializar FastAPI
app = FastAPI()

# Cargar modelo y datos
modelo_knn = joblib.load('modelo_knn.pkl')
df_binarias = pd.read_csv('data_preprocesada.csv')

class Consulta(BaseModel):
    horario: str
    condiciones_binarias: List[int]
    lat: float
    lon: float

@app.get("/recomendar_locales")
def recomendar_locales(horario: str = Query(...), condiciones_binarias: List[int] = Query(...), lat: float = Query(...), lon: float = Query(...)):
    # Crear vector de consulta
    consulta = pd.Series([horario] + condiciones_binarias)
    consulta = consulta.values.reshape(1, -1)
    
    # Encontrar vecinos más cercanos
    distancias, indices = modelo_knn.kneighbors(consulta)
    recomendaciones = df_binarias.iloc[indices[0]]
    
    # Ordenar por num_of_reviews y avg_rating
    recomendaciones = recomendaciones.sort_values(by=['num_of_reviews', 'avg_rating'], ascending=[False, False])
    return recomendaciones[['name', 'avg_rating', 'num_of_reviews', 'latitude', 'longitude','zip_code']].to_dict(orient='records')
